# Qubit relaxation

Qubits decay with time. Let's run a program that measures the qubit relaxation time, also named T1.

In [ ]:
from pyquil.gates import X, I, MEASURE
from pyquil.quil import Program
from pyquil.api import QVMConnection, QPUConnection

We're going to create a list of programs, each elongated with a number of identity (I) gates.

In [ ]:
import numpy as np

qubits = [0] # Which qubit(s) to measure.
start_time = 0.5 # Start time in microseconds.
max_delay = 45. # Stop time in microseconds.
num_pts = 18 # Number of data points.
wait_time = 1 # Each identity gate waits for 1us on the QVM.

delays = np.linspace(start_time, max_delay, num_pts)
progs = []

for delay in delays:
    p = Program()
    p.inst([X(qq) for qq in qubits])
    num_ids = int(round(delay / wait_time, 0))
    for i in range(num_ids):
        p.inst([I(qq) for qq in qubits])
    progs.append(p)

### Run on the QVM with noise simulation

Our QVM, or quantum virtual machine, can simulate noise. For this, run the `batch_run_noisy_qvm` function that is added in this repository.

In [ ]:
from noisy_qvm import batch_run_noisy_qvm

Connect to the QVM and run the programs.

In [ ]:
qvm = QVMConnection()

You can play with the damping to change the relaxation time of the virtual qubit.

In [ ]:
data = np.array(batch_run_noisy_qvm(progs, qvm, trials=500, damping=0.05))

Plot the data!

In [ ]:
for q in qubits:
    %pylab inline
    x = delays
    y = data.mean(axis=1).T[q]
    yerr = data.var(axis=1)
    
    plt.figure()
    plt.plot(x, y, '-o')
    plt.xlabel("t (us)")
    plt.ylabel("Qubit state")

We can fit this data to an exponential decay model.
For this we can use the lmfit library.

In [ ]:
from lmfit.model import Model

def ir_func(x, amplitude, baseline, t1):
    return baseline + amplitude * exp(-x / t1)

a_guess=1
b_guess=0
t1_guess = 15

ir_model = Model(ir_func)
fit = ir_model.fit(y.T, x=x, amplitude=a_guess, baseline=b_guess, t1=t1_guess)
fit.plot()
plt.xlabel("t (us)")
plt.ylabel("Qubit state")

fit.params["t1"].value

### Run on the QPU

If you have received QPU access, you can run this measurement on the QPU! For that, we need to connect to the QPU first.

In [ ]:
qpu = QPUConnection("8Q-Agave")

Now, create the programs. You can do this for one or more qubits.

In [ ]:
import numpy as np

qubits = [0] # Which qubit(s) to measure.
start_time = 0.5 # Start time in microseconds.
max_delay = 20. # Stop time in microseconds.
num_pts = 18 # Number of data points.
wait_time = 50e-3 # Each identity gate waits for 50 nanoseconds on the QPU.

delays = np.linspace(start_time, max_delay, num_pts)
progs = []

for delay in delays:
    p = Program()
    p.inst([X(qq) for qq in qubits])
    num_ids = int(round(delay / wait_time, 0))
    for i in range(num_ids):
        p.inst([I(qq) for qq in qubits])
    progs.append(p)

Run the programs and gather the data.

In [ ]:
data = []

for program in progs:
    data.append(qpu.run_and_measure(program, qubits, trials=500, needs_compilation=False))
    
data = np.array(data)

Plot the data.

In [ ]:
for q in qubits:
    x = delays
    y = data.mean(axis=1).T[qubits.index(q)]

    ir_model = Model(ir_func)
    fit = ir_model.fit(y, x=x, amplitude=a_guess, baseline=b_guess, t1=t1_guess)
    t1 = fit.params["t1"].value

    fit.plot()
    plt.xlabel("t (us)")
    plt.ylabel("Qubit state")
    plt.title(f"Qubit {q}, T1 = {np.round(t1, 2)}")